In [1]:
import os

import pandas as pd
import geopandas as gpd

import numpy as np
from geopy.geocoders import Nominatim # get location coordinates
import folium
from folium import plugins

In [2]:
wd = '/mnt/cabinaData/openmind/cv_maps' # ruta absoluta para evitar problemas por desplazamiento

Importamos el csv como dataframe. 

In [3]:
df = pd.read_csv(os.path.join(wd,"data","MAP_INFO_GENERAL_RESUMEN_DEP.csv"), sep =';', dtype={'DEPARTAMENTO':str})

In [4]:
df.head()

,DEPARTAMENTO,n_CR,n_DX,n_CT,n_KO
0,01,124.0,35.0,4.0,NaN
1,02,329.0,983.0,50.0,1.0
2,03,26.0,68.0,6.0,NaN
3,04,1.0,4.0,NaN,NaN
4,05,NaN,NaN,NaN,NaN


Importamos el fichero GeoJSON como dataframe gracias a la libreria `geopandas`.

In [5]:
limits = gpd.read_file(os.path.join(wd,"data","departamentos_salud_ogr.json"))

In [6]:
limits.head()

,DPTOCRC,NOMBRE,DPTO_KEY,geometry
0,01,VINAROS,2005051712223793901,"POLYGON ((-0.23753 40.69794, -0.23356 40.70855..."
1,02,CASTELLON,2005051712223793902,"MULTIPOLYGON (((0.14852 40.08646, 0.14820 40.0..."
2,03,LA PLANA,2005051712223793903,"POLYGON ((-0.30857 39.79699, -0.31603 39.79983..."
3,04,SAGUNTO,2005051712223793904,"POLYGON ((-0.25889 39.61493, -0.25969 39.61455..."
4,05,VALENCIA - CLINICO,2005051712223793905,"POLYGON ((-0.36789 39.60005, -0.36721 39.60501..."


A los ID de Departamentos y zonas de nuestro csv sintetico con un solo digito les falta un 0 a la izquierda. Se lo anyadimos con apply.

In [7]:
df['DPTOCRC'] = df['DEPARTAMENTO'].apply(lambda x: '0' + x if len(x) == 1 else x)

In [8]:
#df = df.drop([0]) # elimino la fila 0

In [9]:
df.head()

,DEPARTAMENTO,n_CR,n_DX,n_CT,n_KO,DPTOCRC
0,01,124.0,35.0,4.0,NaN,01
1,02,329.0,983.0,50.0,1.0,02
2,03,26.0,68.0,6.0,NaN,03
3,04,1.0,4.0,NaN,NaN,04
4,05,NaN,NaN,NaN,NaN,05


Unimos los dos dataframes usando el codigo de cuatro digitos de la zona como clave primaria

In [10]:
limits = limits.merge(df, how='inner', on='DPTOCRC')
#limits = df

In [11]:
limits.head()

,DPTOCRC,NOMBRE,DPTO_KEY,geometry,DEPARTAMENTO,n_CR,n_DX,n_CT,n_KO
0,01,VINAROS,2005051712223793901,"POLYGON ((-0.23753 40.69794, -0.23356 40.70855...",01,124.0,35.0,4.0,NaN
1,02,CASTELLON,2005051712223793902,"MULTIPOLYGON (((0.14852 40.08646, 0.14820 40.0...",02,329.0,983.0,50.0,1.0
2,03,LA PLANA,2005051712223793903,"POLYGON ((-0.30857 39.79699, -0.31603 39.79983...",03,26.0,68.0,6.0,NaN
3,04,SAGUNTO,2005051712223793904,"POLYGON ((-0.25889 39.61493, -0.25969 39.61455...",04,1.0,4.0,NaN,NaN
4,05,VALENCIA - CLINICO,2005051712223793905,"POLYGON ((-0.36789 39.60005, -0.36721 39.60501...",05,NaN,NaN,NaN,NaN


Podemos obtener las coordenadas de un punto central de la comunidad valenciana usando el geolocalizador `Nominatim`, de libre uso, contenido en la librería `Geopy`.

In [12]:
# Get geographical coordinate of CV
address = 'Comunitat Valenciana'
geolocator = Nominatim(user_agent="ceib")

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Comunitat Valenciana are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Comunitat Valenciana are 39.6819591, -0.7654406.


### Generacion del mapa

En primero lugar, se genera el mapa centrado en las coordenadas indicadas y con un zoom inicial definido. Se indica que no anyada tiles inicialmente, asi podremos tener control de todo lo que se incluye en el mapa. Despues se le anyade la tile layer "Light Map" `CartoDB positron`, porque es de libre acceso y muy aseptica a nivel visual. Ademas, se incluye un plugin para poder visualizar el mapa a pantalla completa.

In [14]:
# create a plain world map Mapbox Bright ,'CartoDB positron'
m = folium.Map(location=[latitude, longitude], zoom_start=8, tiles=None)
folium.TileLayer(tiles='OpenStreetMap',name="Light Map",control=False).add_to(m)

plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(m)

Se anyade la capa del mapa coropletico, que se colorea en funcion de una determinada variable. Para cada campo se incluye una explicacion.

In [15]:
folium.Choropleth(
 geo_data=limits, # Objeto en el cual estan incluidas las coordenadas
 name='Nº CR', # Nombre de la capa
 data=limits, # Objeto en el cual se encuentra la variable a representar. En este caso coincide que el objeto es el mismo
              # para la variable y las coordenadas, aunque podia no ser el caso.
 columns=['DPTOCRC','n_CR'], # En primer lugar se indica la columna con la clave que relaciona
                                    # En segundo, la variable a representar
 key_on="feature.properties.DPTOCRC", # Localizacion del campo DEPTOCRC en el GeoJSON
 fill_color='YlGnBu', # Codigo de la paleta de colores
 fill_opacity=0.8, # Opacidad del relleno
 line_opacity=0.5, # Opacidad de los nombres
 legend_name='Numero de casos' # titulo de la leyenda
).add_to(m)


Se anyaden las style y highlight functions para resaltar una zona al pasar el raton por encima. Ademas, se incorporan cajas de texto que indican el valor de las variables que se quiera por zona senyalada. Por ultimo, se incorpora un control de capas para poder activar/desactivar los rellenos.

In [ ]:
m.add_children(plugins.HeatMap(stationArr, radius=15))

In [21]:
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    limits, # Dataframe con la variable a representar
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip( # Caja de texto
        fields=['DPTOCRC','NOMBRE','n_CR'], # Variables incluidas
        aliases=['Dep. de salud: ','Nombre: ','Nº CR: '], # Texto que precede a las variables
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(NIL) # se anyade al mapa
m.keep_in_front(NIL) # se lleva al frente para no quedar detras del mapa
folium.map.LayerControl().add_to(m) # control de capas para poder ocultar el relleno

In [17]:
m

iopub data rate exceeded.  The
                    notebook server will temporarily stop sending iopub
                    messages to the client in order to avoid crashing it.
                    To change this limit, set the config variable
                    `--NotebookApp.iopub_data_rate_limit`.

In [68]:
m.save(os.path.join(wd,"results","Departamentos.html"))

In [69]:
! pwd

/mnt/cabinaData/openmind/cv_maps
